In [1]:
import sys
sys.path.append('../../')

In [7]:
from codes.docs.analysis import data_preprocessing, MLtraining
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as pyplot
import pandas as pd
import tqdm

%matplotlib inline

from sklearn.linear_model import LinearRegression

In [8]:
european_diffusion_dataset_term=pd.read_csv('./preprocessed_data/diffusion/term/european_diffusion_dataset_term.csv')
def calc_zero_proportion(column):
    return(column==0).sum()/len(column)

zero_proportions_features=european_diffusion_dataset_term.loc[:,'PreCG.R_PreCG.L':'ITG.R_ITG.L'].agg(calc_zero_proportion)

print(f'removed {(zero_proportions_features>0).sum()} features, in which at least 1 of subjects registered zeros values ')
european_diffusion_dataset_term_reduced=european_diffusion_dataset_term.drop(columns=zero_proportions_features.index[zero_proportions_features>0])
#european_diffusion_dataset_term_reduced is the reduced version with non-zero features

removed 1492 features, in which at least 1 of subjects registered zeros values 


In [9]:
#preprocess the PRS score
PRS_thresholds=['PRS_1e08', 'PRS_1e07', 'PRS_1e06', 'PRS_1e05', 'PRS_0.0001','PRS_0.001', 'PRS_0.01', 'PRS_0.05', 'PRS_0.1', 'PRS_0.5', 'PRS_1']


adjusted_prs_score=data_preprocessing.adjusting_for_covariates_with_lin_reg(np.array(european_diffusion_dataset_term_reduced[PRS_thresholds]),
np.array(european_diffusion_dataset_term_reduced['Anc_PC1']),np.array(european_diffusion_dataset_term_reduced['Anc_PC2']),np.array(european_diffusion_dataset_term_reduced['Anc_PC3']))

In [384]:
X=np.asarray(european_diffusion_dataset_term_reduced.iloc[:,203:2716])
y=np.asarray(european_diffusion_dataset_term_reduced['PMA_diff'])

In [512]:
def remove_correlated_features(X,target,threshold=0.8):
    X=X.copy() # avoid changes to the original array
    dict_of_features=defaultdict(list)
    for i in range(X.shape[1]):
        dict_of_features[i].append(i)
    
    while True:
        corr_matrix_between_features=np.corrcoef(X,rowvar=False) # calculate the correlation matrix between features
        corr_matrix_between_features[np.isnan(corr_matrix_between_features)]=0 # set any nan number to 0.
        np.fill_diagonal(corr_matrix_between_features,0) # fill the diagonal of 1s to 0s.
        maximum_correlation=np.max(np.abs(corr_matrix_between_features))
        if maximum_correlation < threshold:
            break
        # for some reason, when the nrow (X) is < ncol(X), then the resulting correlation matrix is symmetrical, but not when nrow(X) is less than ncol(X), the difference in the non-symmetrical is very small, almost non-negligible. However, when you run np.where, this will result in differing output.
        try:
            feature_0,feature_1=np.where(np.abs(corr_matrix_between_features)==maximum_correlation)[0] # find where the highest correlation pair is
        except ValueError: # when the unpacking is unsuccessful, feature is an array int
            feature_0,feature_1=np.where(np.abs(corr_matrix_between_features)==maximum_correlation)
            feature_0=feature_0[0] #get it into an interger format to be consistent
            feature_1=feature_1[0]
        feature_0_corr_coef=pearsonr(X[:,feature_0],target)[0]
        feature_1_corr_coef=pearsonr(X[:,feature_1],target)[0]
        if feature_0_corr_coef>feature_1_corr_coef: # check which one has higher correlation to the target
            dict_of_features[feature_0].extend(dict_of_features[feature_1]) # represent the lower correlated feature with the higher one.
            X[:,feature_1]=np.nan # set the values in the eliminated feature as nan. so that next time, we don't have to recompute the correlation.
            del dict_of_features[feature_1] # delete that key
        else:
            dict_of_features[feature_1].extend(dict_of_features[feature_0])
            X[:,feature_0]=np.nan
            del dict_of_features[feature_0]

    return dict_of_features

def merge_the_features(X,dict_of_features,average=False):
    """
    merge the correlated features, either by elimination or averaging
    if elimination: just choose the key of the dictionary
    if average: average the features within each keys.
    """
    X=X.copy()
    if average:
        for key,values in dict_of_features.items():
            if len(values)>1:
                X[:,key]=np.mean(X[:,values],axis=1)

    for col in range(X.shape[1]):
        if not dict_of_features[col]:#if this key is empty
            X[:,col]=np.nan
    return X
    
def merge_features_names(dict_of_features,feature_names):
    """
    Used in conjunction with the merge the features to get the name of the representative feature name and its constituents.
    Output:
        defaultdict(list,
            {'SFGdor.R_PreCG.L': ['SMA.R_PreCG.L'],
             'ORBsup.L_SFGdor.R': ['SFGmed.R_ORBsup.L'],
             'ORBsup.R_PreCG.R': ['ORBmid.R_PreCG.R'],...}
    """
    new_dict_of_names=defaultdict(list)
    for key,values in dict_of_features.items():
        if len(values)>1:
            new_dict_of_names[feature_names[key]]=list(feature_names[values[1:]])
    return new_dict_of_names

In [403]:
%timeit remove_correlated_features(X,adjusted_prs_score[:,7])

29.9 s ± 1.69 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [467]:
average_X=merge_the_features(X,test,average=True)
eliminate_X=merge_the_features(X,test,average=False)

In [471]:
average_X=pd.DataFrame(average_X)

In [475]:
average_X.columns=european_diffusion_dataset_term_reduced.iloc[:,203:2716].columns

In [480]:
average_X.dropna(axis=1)

,PreCG.R_PreCG.L,SFGdor.L_PreCG.L,SFGdor.R_PreCG.L,SFGdor.R_PreCG.R,ORBsup.L_PreCG.L,ORBsup.L_SFGdor.L,ORBsup.L_SFGdor.R,ORBsup.R_PreCG.L,ORBsup.R_PreCG.R,ORBsup.R_SFGdor.R,...,ITG.R_PUT.R,ITG.R_PAL.L,ITG.R_PAL.R,ITG.R_THA.L,ITG.R_THA.R,ITG.R_HES.R,ITG.R_STG.R,ITG.R_TPOsup.R,ITG.R_MTG.R,ITG.R_TPOmid.R
0,0.009573,3.672285,0.012784,3.640346,0.014490,0.558478,0.009154,0.000045,0.011890,0.528269,...,0.497267,0.002266,0.110881,0.004266,0.398147,0.011649,0.676557,1.597469,10.532588,2.491461
1,0.051696,1.790308,0.050054,2.413920,0.013734,0.423371,0.013725,0.000235,0.016596,0.384222,...,0.379149,0.001185,0.063158,0.005045,0.450001,0.020439,0.934958,1.512014,8.328801,1.951853
2,0.042033,3.778783,0.030113,5.280913,0.008970,0.283879,0.017326,0.000617,0.006661,0.865717,...,0.571502,0.001337,0.067891,0.006284,0.528868,0.016369,0.606910,1.008959,10.403495,2.714284
3,0.088358,2.819123,0.041142,3.580281,0.009090,0.624162,0.007946,0.000474,0.004920,1.331177,...,0.635597,0.005216,0.160275,0.018250,0.646850,0.011186,0.668314,1.654926,9.256667,3.834038
4,0.095817,2.582996,0.074419,3.737281,0.004239,0.708756,0.011278,0.000892,0.006026,0.272395,...,0.437032,0.001131,0.119716,0.004303,0.464031,0.010185,0.522433,0.840888,10.196755,3.360334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.083487,2.869284,0.063264,2.916596,0.011843,0.985391,0.012439,0.001119,0.009370,1.329999,...,0.489779,0.002875,0.112407,0.011803,0.582987,0.007024,0.772592,1.785439,11.115254,2.976883
147,0.030477,1.949638,0.028811,2.857530,0.008456,0.410248,0.008398,0.000535,0.007975,0.770036,...,0.275731,0.001454,0.098533,0.006996,0.663175,0.024217,0.367410,0.795176,8.197528,2.956401
148,0.075496,2.815264,0.103853,3.762986,0.008082,0.634381,0.010858,0.000812,0.008078,0.992587,...,0.525683,0.003362,0.112680,0.018385,0.437035,0.012304,0.631822,0.914461,9.966092,1.549367
149,0.019497,1.943436,0.035241,2.433037,0.007883,0.409369,0.005244,0.000601,0.008509,0.395321,...,0.460397,0.000759,0.096739,0.006009,0.406317,0.015420,0.371186,1.131914,8.176903,2.623584


In [513]:
test2=merge_features_names(test,average_X.columns)